# 02 — Forecast models with conformal intervals

Demonstrates rolling-origin forecasts for one country and split conformal intervals.

In [ ]:
import numpy as np
import pandas as pd
from src.data_utils import load_raw_any, make_supervised
from src.forecast import rolling_origin_forecast, mae
from src.conformal import split_conformal_intervals, apply_conformal, empirical_coverage

raw_dir = '../data/raw'
df = load_raw_any(raw_dir)
df_sup = make_supervised(df, lags=3)

countries = sorted(df_sup['country'].dropna().unique().tolist())
print('Countries:', len(countries))
country = countries[0]
print('Example country:', country)

trues, preds = rolling_origin_forecast(df_sup, country, min_train=10)
print('MAE:', mae(trues, preds))

n = len(trues)
k = max(1, n//2)
calib_resid = np.abs(trues[:k] - preds[:k])
q = split_conformal_intervals(calib_resid, alpha=0.1)
lower, upper = apply_conformal(preds, q)
cov = empirical_coverage(trues, lower, upper)
print('Conformal q (alpha=0.1):', q, '| Empirical coverage:', cov)